# Análise Exploratória da Tabela de Requisição de Materiais

Este notebook apresenta a análise exploratória de dados da tabela de requisição de materiais. o principal intuito desta análise é entender melhor os dados relacionados a materiais e a partir disto gerar hipóteses.

Nesse sentido, aqui serão examinadas as principais características de um conjunto de dados, identificando padrões, tendências, outliers e possíveis relações entre variáveis. Utilizando estatísticas descritivas, gráficos e visualizações, a AED ajudará a entender a estrutura dos dados, verificar hipóteses e orientar etapas futuras, como modelagem ou limpeza. Essa análise é essencial para fazer escolhas informadas sobre quais métodos e técnicas aplicar no restante do projeto.

In [ ]:
%pip install klib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import klib

In [ ]:
df = pd.read_csv('sample_data/tabela_big_data_ft_falhas.csv', encoding='latin-1', delimiter='\t', on_bad_lines='skip')

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns=lambda x: x.replace("'", "").replace('"', ""))

In [ ]:
df

In [ ]:
columns_to_drop = ['TEMPO_ATENDIMENTO_2','TEMPO_INDISPONIVEL_2','TEMPO_LIBERACAO_2','TEMPO_REPARO_LIB_2','TEMPO_REPARO_TOTAL_2','DIM_ATUACAO_SK']

df_cleaned = df.drop(columns=columns_to_drop)

In [ ]:
df_cleaned = df_cleaned.dropna()

In [ ]:
df_nodup = df_cleaned.drop_duplicates()

In [ ]:
df_nodup

In [ ]:
df_nodup.reset_index(drop=True, inplace=True)

In [ ]:
import pandas as pd
import polars as pl

df_nodup = df_nodup.astype(str)

pl_df = pl.from_pandas(df_nodup)

numeric_columns = []
for col in pl_df.columns:
    sample_values = pl_df[col].head(100)
    sample_values = sample_values.str.strip_chars("'\"")
    sample_values = sample_values.str.replace_all(',', '.')
    sample_floats = sample_values.cast(pl.Float64, strict=False)
    num_non_null = sample_floats.drop_nulls().len()
    num_total = sample_floats.len()
    if num_total > 0 and (num_non_null / num_total) >= 0.8:
        numeric_columns.append(col)

for col in numeric_columns:
    pl_df = pl_df.with_columns(
        pl.col(col).str.strip_chars("'\"").alias(col)
    )

    pl_df = pl_df.with_columns(
        pl.col(col).str.replace_all(',', '.').alias(col)
    )

    pl_df = pl_df.with_columns(
        pl.col(col).cast(pl.Float64, strict=False).alias(col)
    )

    pl_df_non_null = pl_df[col].drop_nulls()
    if len(pl_df_non_null) == 0:
        continue

    is_integer = (pl_df_non_null % 1 == 0) | (pl_df_non_null.is_null())
    if is_integer.all():
        pl_df = pl_df.with_columns(
            pl.col(col).cast(pl.Int64, strict=False).alias(col)
        )

df_cleaned = pl_df.to_pandas()

print(df_cleaned)


In [ ]:
df_cleaned

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

falhas_por_equipamento = df_cleaned['ID_EQUIPAMENTO'].value_counts().head(10)

falhas_por_equipamento.plot(kind='bar', figsize=(10,6))
plt.title('Top 10 Equipamentos com Mais Falhas')
plt.xlabel('ID do Equipamento')
plt.ylabel('Número de Falhas')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df_cleaned['Data_Falha'] = pd.to_datetime(df_cleaned['ID_DT_FALHA'], format='%Y%m%d')

tempo_indisponivel_por_data = df_cleaned.groupby('Data_Falha')['TEMPO_INDISPONIVEL'].sum()

tempo_indisponivel_por_data.plot(figsize=(12,6))
plt.title('Tempo Indisponível por Data de Falha')
plt.xlabel('Data da Falha')
plt.ylabel('Tempo Indisponível (Soma)')
plt.show()

In [ ]:
falhas_por_localidade = df_cleaned['LOCALIDADE'].value_counts().head(10)

falhas_por_localidade.plot(kind='bar', figsize=(10,6))
plt.title('Top 10 Localidades com Mais Falhas')
plt.xlabel('Localidade')
plt.ylabel('Número de Falhas')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import seaborn as sns

sns.scatterplot(data=df_cleaned, x='NR_TEMPO_ACESSO', y='TEMPO_INDISPONIVEL')
plt.title('Tempo Indisponível vs. Tempo de Acesso')
plt.xlabel('Tempo de Acesso')
plt.ylabel('Tempo Indisponível')
plt.show()


In [ ]:
colunas_numericas = ['TEMPO_LIBERACAO', 'TEMPO_INDISPONIVEL', 'NR_TEMPO_ACESSO']

df_numerico = df_cleaned[colunas_numericas]

matriz_corr = df_numerico.corr()

plt.figure(figsize=(8,6))
sns.heatmap(matriz_corr, annot=True, cmap='coolwarm')
plt.title('Heatmap de Correlação')
plt.show()


In [ ]:
contingencia = pd.crosstab(df_cleaned['ID_EQUIPAMENTO'], df_cleaned['TX_GARANTIA'])

top_equipamentos = df_cleaned['ID_EQUIPAMENTO'].value_counts().head(10).index
contingencia_top = contingencia.loc[top_equipamentos]

contingencia_top.plot(kind='bar', stacked=True, figsize=(12,8))
plt.title('Falhas por Equipamento e Tipo de Garantia')
plt.xlabel('ID do Equipamento')
plt.ylabel('Número de Falhas')
plt.legend(title='Tipo de Garantia')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df_cleaned['Ano_Falha'] = df_cleaned['Data_Falha'].dt.year

falhas_por_ano = df_cleaned['Ano_Falha'].value_counts().sort_index()

falhas_por_ano.plot(kind='line', marker='o', figsize=(10,6))
plt.title('Número de Falhas por Ano')
plt.xlabel('Ano')
plt.ylabel('Número de Falhas')
plt.show()


In [ ]:
sns.boxplot(data=df_cleaned, x='TX_GARANTIA', y='TEMPO_INDISPONIVEL')
plt.title('Tempo Indisponível por Tipo de Garantia')
plt.xlabel('Tipo de Garantia')
plt.ylabel('Tempo Indisponível')
plt.xticks(rotation=45)
plt.show()


In [ ]:
media_tempo_por_equipamento = df_cleaned.groupby('ID_EQUIPAMENTO')['TEMPO_INDISPONIVEL'].mean().sort_values(ascending=False).head(10)

media_tempo_por_equipamento.plot(kind='bar', figsize=(10,6))
plt.title('Média do Tempo Indisponível por Equipamento (Top 10)')
plt.xlabel('ID do Equipamento')
plt.ylabel('Tempo Indisponível (Média)')
plt.xticks(rotation=45)
plt.show()


### Hipóteses para Análise de Requisição de Materiais

1. **A Falta de Materiais Críticos Aumenta o Tempo de Indisponibilidade dos Equipamentos**:
   - Hipótese: A ausência de certos materiais essenciais pode causar atrasos significativos no processo de reparo, prolongando o tempo de indisponibilidade dos equipamentos. Esta hipótese pode ser testada analisando o impacto de materiais indisponíveis nos tempos de reparo e atendimento.

2. **Problemas na Gestão de Estoque Contribuem para Atrasos no Atendimento de Falhas**:
   - Hipótese: A gestão ineficaz de estoque, como o estoque insuficiente de peças de reposição, pode resultar em atrasos no atendimento de falhas. Analisar a correlação entre a quantidade de itens em estoque e a frequência de falhas não atendidas ou demoradas pode ajudar a validar essa hipótese.

3. **Demandas de Requisição Não Planejadas Afetam a Programação de Manutenções**:
   - Hipótese: Requisições de materiais não planejadas impactam a programação de manutenções preventivas, desviando recursos e materiais para atendimentos emergenciais. Esta hipótese pode ser avaliada analisando o impacto das requisições de última hora nos cronogramas de manutenção.

4. **Locais com Alta Frequência de Falhas Requerem Estoque de Materiais Específicos**:
   - Hipótese: Em locais com maior volume de falhas, é necessário manter um estoque local de peças frequentemente requisitadas para reduzir o tempo de atendimento e reparo. Esta hipótese pode ser investigada ao relacionar a frequência de falhas por localidade com os materiais disponíveis em estoque.

5. **Equipamentos Fora de Garantia Demandam Mais Requisições de Materiais**:
   - Hipótese: Equipamentos fora da garantia podem ter uma necessidade maior de peças de reposição, o que aumenta a demanda por requisições de materiais. Analisar a relação entre o status de garantia dos equipamentos e a quantidade de materiais requisitados pode fornecer insights sobre essa hipótese.

6. **Atrasos nas Requisições de Materiais Afetam o Cumprimento dos Prazos de Reparo**:
   - Hipótese: Processos demorados para a aprovação e liberação de requisições de materiais impactam negativamente os prazos de reparo, gerando mais tempo de equipamento inativo. Para validar essa hipótese, pode-se observar a relação entre o tempo de aprovação das requisições e o tempo total de reparo.


In [ ]:
!pip install pyarrow


In [ ]:
df_nodup.to_parquet('seu_arquivo.parquet', engine='pyarrow', index=False)


In [ ]:
from google.colab import files
files.download('seu_arquivo.parquet')